In [ ]:
import circuitgraph as cg
from circuitsim import CircuitSimulator
# import pyqbf
# from pyqbf.formula import PCNF #, split_atoms_and_auxvars
# from pyqbf.solvers import Solver
import random
import numpy as np
import math
import pickle
import os

In [ ]:
with open("affine_lib.pkl", "rb") as f:
  affine_lib = pickle.load(f)

In [ ]:
def sort_pins(pins):
  l_pins = list(pins)
  l_pins.sort()
  return l_pins

def bmc_step(v_file_verify, v_file_gold, v_file_trans, n_timeframe):
    # how to fix the connection?

    # Load the gate-level circuit from Verilog
    circuit_verify = cg.from_file(v_file_verify)
    circuit_gold = cg.from_file(v_file_gold)
    circuit_trans = cg.from_file(v_file_trans)

    ver_in = sort_pins(circuit_verify.inputs())
    # print(ver_in)
    gold_in = sort_pins(circuit_gold.inputs())
    # print(gold_in)
    trans_out = sort_pins(circuit_trans.outputs())
    # print(trans_out)
    trans_in = sort_pins(set(filter(lambda x: "in" in x, circuit_trans.inputs())))
    # print(trans_in)
    trans_PI = sort_pins(set(filter(lambda x: "in" not in x, circuit_trans.inputs())))
    # print(trans_PI)

    circuit = cg.Circuit()
    for i in range(len(circuit_verify.inputs())):
        circuit.add("PI_{}".format(i + 1), "input")
    circuit_PI = sort_pins(circuit.inputs().copy()) # only contain PI_
    # print(circuit_PI)

    # ----
    if (n_timeframe == 0):
      connection = dict(zip(ver_in, circuit_PI))
      # print(connection)
    else:
      for i in range(n_timeframe):
        for j in range(len(trans_PI)):
          circuit.add("Ctrl_PI_{}_{}".format(i, j), "input")
        if (i == 0):
          connection = dict(zip(trans_in, circuit_PI))
          connection.update(dict(zip(trans_PI, ["Ctrl_PI_{}_{}".format(i, j) for j in range(len(trans_PI))])))
          circuit.add_subcircuit(circuit_trans, "Trans_{}".format(i), connections = connection)
        else:
          connection = dict(zip(trans_in, ["Trans_{}_{}".format(i - 1, o) for o in trans_out]))
          connection.update(dict(zip(trans_PI, ["Ctrl_PI_{}_{}".format(i, j) for j in range(len(trans_PI))])))
          circuit.add_subcircuit(circuit_trans, "Trans_{}".format(i), connections = connection)
        # print(connection)
      connection = dict(zip(ver_in, ["Trans_{}_{}".format(n_timeframe - 1, o) for o in trans_out]))
      # print(connection)
    circuit.add_subcircuit(circuit_verify, "VER", connections = connection)
    connection = dict(zip(gold_in, circuit_PI))
    circuit.add_subcircuit(circuit_gold, "GOLD", connections = connection)
    circuit.add("miter", "xor", fanin = ["VER_" + next(iter(circuit_verify.outputs())), "GOLD_"+next(iter(circuit_gold.outputs()))], output = True)
    # print(connection)
    cg.to_file(circuit, "miter.v")

    cnf = cg.sat.cnf(circuit)
    obj2id = {
        "PI": [],
        "miter": [],
        "Ctrl_PI": [],
        "Inner": []
    }
    for k, v in cnf[1].id2obj.items():
        if "PI" == v[0:2]:
            obj2id["PI"].append(k)
        elif "miter" == v:
            obj2id["miter"].append(k)
        elif ("Ctrl_PI" == v[0:7]):
            obj2id["Ctrl_PI"].append(k)
        else:
            obj2id["Inner"].append(k)
    # cnf[0].append(clause = [cnf[1].id2obj.get()])
    # Output CNF clauses
    # print(cnf)
    cg.sat.add_assumptions(cnf[0], cnf[1], {'miter': 0})
    cnf[0].to_file("miter.cnf")

    # return cnf[1].id2obj
    return obj2id, trans_PI, cnf[1].id2obj

# bmc_step("cir1.v", "cir2.v", "ctrlckt_3.v", 1)
# obj2id


In [ ]:
def get_cut(circuit, nd = None):
  # circuit = cg.from_file(v_file)
  # ckt_in = sort_pins(circuit.inputs())
  # ckt_out = sort_pins(circuit.outputs())
  cut_info = dict()
  if (nd == None):
    for n in circuit.nodes():
      cut_info[n] = circuit.kcuts(n, 4)
      # cut_info[n].sort(key = lambda x: min(len(p) for p in circuit.paths(x, n)))
  else:
    for n in nd:
      cut_info[n] = circuit.kcuts(n, 4)
      # cut_info[n].sort(key = lambda x: min(len(p) for p in circuit.paths(x, n)))
  # print(cut_info)
  return cut_info

def get_subcircuit(circuit, fanin, fanout):
  subcircuit = cg.Circuit()
  tracklist = set()
  for nin in fanin:
    for nout in fanout:
      # print(*circuit.paths(nin, nout))
      for path in circuit.paths(nin, nout):
        if (len(fanin & set(path)) > 1): continue
        tracklist |= set(path)
  return cg.tx.subcircuit(circuit, tracklist, modify_io = True)

def get_truth(circuit):
  # print(circuit.nodes())
  simulator = CircuitSimulator(circuit.copy())
  vectors = []
  # truth = ""
  inpin = sort_pins(circuit.inputs())
  num_in = len(inpin)
  for vec in range(2**num_in):
    vectors.append({inpin[i]: (True if (j == '1') else False) for i, j in enumerate(format(vec, f'0{num_in}b'))})
    # vectors.append({i: random.choice([True, False]) for i in circuit.inputs()})
  sim_result = simulator.simulate(vectors)
  opin = list(sim_result[0].keys())
  table = dict()
  for vec in range(2**num_in):
    table[format(vec, f'0{num_in}b')] = [sim_result[vec][opin[i]] for i in range(len(opin))]
  return table, (inpin, opin)
  # table = np.zeros((len(inpin)+len(opin), 2**num_in))

def remove_buf(ckt):
  for nd in ckt.copy().nodes():
    if (ckt.type(nd) == "buf" and not ckt.is_output(nd)):
      fin = ckt.fanin(nd)
      fos = ckt.fanout(nd)
      if (ckt.type(fin) != 'input'): continue
      ckt.remove(nd)
      for fo in fos:
        # print(fin, fo)
        ckt.connect(fin, fo)

def io_buffing(ckt):
  for i, po in enumerate(ckt.outputs()):
    ckt.set_output(po, False)
    ckt.add("PO_{}".format(i), "buf", fanin = [po], output = True)
  for i, pi in enumerate(ckt.inputs()):
    ckt.set_type(pi, "buf")
    ckt.add("PI_{}".format(i), "input", fanout = [pi])

def single_fanout(ckt):
  # can be modify by limit_fanout?
  nodes = [*ckt.topo_sort()]
  # print(nodes)
  cnt = 0
  for nd in nodes:
    if (ckt.type(nd) == 'input'):
      for fo in ckt.fanout(nd).copy():
        ckt.disconnect(nd, fo)
        ckt.add("dup{}_{}".format(cnt, nd), "buf", fanin=nd, fanout=fo)
        cnt+=1
    if (len(ckt.fanout(nd)) > 1):
      # print(nd, ckt.startpoints(nd))
      cp_cone = get_subcircuit(ckt, ckt.startpoints(nd), [nd])
      # cg.to_file(cp_cone, "sub.v")
      cp_cone.set_output(nd)
      flag = True
      for fo in ckt.fanout(nd).copy():
        # print(nd, "/", ckt.fanout(nd))
        if (flag):
          flag = False
          # print(nd, "/", ckt.fanout(nd))
          continue
        else:
          ckt.disconnect(nd, fo)
          # print(sort_pins(cp_cone.inputs()), sort_pins(ckt.startpoints(nd)))
          connect = dict(zip(sort_pins(cp_cone.inputs()), sort_pins(ckt.startpoints(nd))))
          connect[nd] = fo
          ckt.add_subcircuit(cp_cone, "dup{}".format(cnt), connections = connect)
          cnt+=1
          # print(nd, "/", ckt.fanout(nd))
def merge_fanout(ckt):
  nodes = [*ckt.topo_sort()]
  for nd in nodes:
    org_nd = nd
    state = False
    while ("dup" in org_nd or state != False):
      if (org_nd[0:3] == "dup" and state == False):
        org_nd = org_nd[3:]
        state = True
      elif (state == True):
        if (org_nd[0] in "0123456789"): org_nd = org_nd[1:]
        else:
          org_nd = org_nd[1:]
          state = False

    if (org_nd != nd):
      if (org_nd not in ckt.nodes()):
        ckt.relabel({nd: org_nd})
      else:
        fos = ckt.fanout(nd)
        for fo in fos:
          ckt.disconnect(nd, fo)
          ckt.connect(org_nd, fo)


In [ ]:
def truth_to_str(truth, check_o = 0):
  '''format: 0000->1111, [in_0][in_1][in_2][in_3]'''
  tr_str = ""
  n = int(math.log2(len(truth)))
  for vec in range(2**n):
    vec_b = format(vec, f'0{n}b')
    val = truth[vec_b]
    tr_str += "1" if val[check_o] else "0"
  return tr_str

def to_ABC_truth(trs):
  '''format: 1111->0000, [in_3][in_2][in_1][in_0] <-> dcba'''
  n = int(math.log2(len(trs)))
  return "".join(map(lambda i: trs[i], [int(format(i, f'0{n}b')[::-1], 2) for i in range(len(trs))]))[::-1]

def cut_to_truth(ckt, cut, fo):
  if (list(cut)[0] == fo): return None, None, None
  cone = get_subcircuit(ckt, cut, [fo])
  i = 0
  while (i + len(cut) < 4):
    cone.add('dummy{}'.format(i), 'input')
    i += 1
  truth, (inpin, _) = get_truth(cone)
  while (i > 0):
    cone.remove('dummy{}'.format(i - 1))
    i -= 1
  return truth, inpin, cone

def check_sig(truth, check_o = 0):
  # print(len(truth))
  num_in = int(math.log2(len(truth)))
  walsh = []
  auto = []
  onset = 0
  for w in range(2**num_in):
    walsh.append(0)
    auto.append(0)
    for x in range(2**num_in):
      w_is_neg = 1 if truth[format(x, f'0{num_in}b')][check_o] else 0
      a_is_neg = w_is_neg
      w_is_neg += str(bin(w & x)).count('1')
      a_is_neg += 1 if truth[format(w ^ x, f'0{num_in}b')][check_o] else 0
      walsh[-1] += -1 if (w_is_neg % 2) else 1
      auto[-1] += -1 if (a_is_neg % 2) else 1
    onset += 1 if truth[format(w, f'0{num_in}b')][check_o] else 0
  return walsh, auto, onset

def check_affine(truth1, truth2):
  '''only work for 4 input truth'''
  if (len(truth1) != 16 or len(truth2) != 16): return False
  walsh1, auto1, onset1 = check_sig(truth1)
  walsh2, auto2, onset2 = check_sig(truth2)
  return (walsh1 == walsh2) and (auto1 == auto2) and (onset1 == onset2 or (onset1 == 16 and onset2 == 0) or (onset1 == 0 and onset2 == 16))

In [ ]:
def circuit_replace(ckt, truth, in_ord, fo, cone):
  '''lib := dict(truth: (truth, ckt))'''
  tr_str = truth_to_str(truth)
  rm_set = cone.nodes() - set(in_ord) - set([fo])
  for fi in in_ord:
    if (ckt.type(fi) != 'input' or "dummy" in fi):
      ckt.disconnect(fi, ckt.fanout(fi))
    else:
      ps = list(ckt.paths(fi, fo))
      # for path in circuit.paths(nin, nout):
      #   if (len(fanin & set(path)) > 1): continue
      if (len(ps) != 1):
        print(ps)
        raise
      if (len(ps[0]) < 2):
        print(ps)
        raise
      ckt.disconnect(fi, ps[0][1])
  for nd in ckt.fanin(fo):
    ckt.disconnect(nd, fo)
  ckt.remove(rm_set)
  ckt.set_type(fo, "buf")

  tr_top = affine_lib[tr_str][0]
  patch = affine_lib[tr_str][1].copy()

  patch_in = sort_pins(patch.inputs())
  # print(patch_in)
  patch_out = sort_pins(patch.outputs())
  # print(patch_out)
  frt_isPO = True
  if (tr_top != tr_str):
    # patch := PI -> PO
    top = affine_lib[tr_top][1].copy()
    for aff_fo in ['PO_0', 'PO_1', 'PO_2', 'PO_3']:
      patch.set_output(aff_fo, False)
    patch.add_subcircuit(top, 'm', connections = {'a': 'PO_0', 'b': 'PO_1', 'c': 'PO_2', 'd': 'PO_3'})
    patch.set_output("m_F0")
    frt_isPO = True
  else:
    # patch := abcd -> FO
    patch.relabel({'a': 'PI_0', 'b': 'PI_1', 'c': 'PI_2', 'd': 'PI_3', 'F0': 'm_F0'})
    frt_isPO = False

  # print(get_truth(patch))
  # if (truth_to_str(get_truth(patch)[0]) != tr_str): raise
  connect = dict(zip(['PI_0', 'PI_1', 'PI_2', 'PI_3'], in_ord))
  connect["m_F0"] = fo
  ckt.add_subcircuit(patch, 'rep_{}'.format(fo), connect)
  frt_id = filter(lambda i: 'dummy' not in in_ord[i] ,[i for i in range(4)])
  return ['rep_{}_P{}_{}'.format(fo, 'O' if frt_isPO else 'I', i) for i in frt_id]

  # affine_tgt = affine_lib[tr_str][1
  # ckt.add_subcircuit(subcircuit, 'aff', dict(zip(affine_tgt.inputs(), c_tgt)))
def map_affine(ckt_tgt, ckt_goal, match_nd):
  '''it would destroy both ckt, please input copy'''
  # Solve Boundary cases!!!
  # return the correct frontier

  (fo_tgt, fo_goal) = match_nd
  cuts_tgt = get_cut(ckt_tgt, [fo_tgt])
  cuts_goal = get_cut(ckt_goal, [fo_goal])
  random.shuffle(cuts_tgt)
  random.shuffle(cuts_goal)
  best_tgt = None
  best_goal = None
  gain = 0
  for c_tgt in cuts_tgt[fo_tgt]:
    for c_goal in cuts_goal[fo_goal]:
      # print(c_tgt, c_goal)
      flag = False
      for nd in c_tgt:
        if ckt_tgt.type(nd) == 'input': flag = True
      for nd in c_goal:
        if ckt_goal.type(nd) == 'input': flag = True
      if (flag): continue
      truth_tgt, ord_tgt, cone_tgt = cut_to_truth(ckt_tgt, c_tgt, fo_tgt)
      truth_goal, ord_goal, cone_goal = cut_to_truth(ckt_goal, c_goal, fo_goal)
      if (truth_tgt == None or truth_goal == None): continue
      if (gain >= len(cone_tgt.nodes()) + len(cone_goal.nodes())): continue
      trstr_tgt = truth_to_str(truth_tgt)
      trstr_goal = truth_to_str(truth_goal)
      if (trstr_tgt == trstr_goal):
        best_tgt = (truth_tgt, ord_tgt, cone_tgt)
        best_goal = (truth_goal, ord_goal, cone_goal)
        gain = len(cone_tgt.nodes()) + len(cone_goal.nodes())
      elif (check_affine(truth_tgt, truth_goal)):
        tgt_cost = 0 if affine_lib[trstr_tgt][0] == trstr_tgt else len(affine_lib[trstr_tgt][1].nodes()) - 8
        goal_cost = 0 if affine_lib[trstr_goal][0] == trstr_goal else len(affine_lib[trstr_goal][1].nodes()) - 8
        gain_aff = len(cone_tgt.nodes()) - tgt_cost + len(cone_goal.nodes()) - goal_cost
        if (gain < gain_aff):
          best_tgt = (truth_tgt, ord_tgt, cone_tgt)
          best_goal = (truth_goal, ord_goal, cone_goal)
          gain = gain_aff

  if (best_tgt == None or best_goal == None): return None
  (truth_tgt, ord_tgt, cone_tgt) = best_tgt
  (truth_goal, ord_goal, cone_goal) = best_goal
  frt_tgt = circuit_replace(ckt_tgt, truth_tgt, ord_tgt, fo_tgt, cone_tgt)
  frt_goal = circuit_replace(ckt_goal, truth_goal, ord_goal, fo_goal, cone_goal)
  return list(zip(frt_tgt, frt_goal)) if truth_to_str(truth_tgt) != truth_to_str(truth_goal) else list(zip(ord_tgt, ord_goal))
        # return list(zip(ord_tgt, ord_goal))
def syn_preprocess(ckt):
  # io_buffing(ckt)
  single_fanout(ckt)
  # print(*filter(lambda x: "dup" in x, ckt.nodes()))
  # print(get_cut(ckt
  remove_buf(ckt)
  ckt.remove_unloaded()
  ckt.add("dummy0", "input")
  ckt.add("dummy1", "input")
  ckt.add("dummy2", "input")

def syn_for_cec(ckt_tgt, ckt_goal):
  # Notice ckt input as frontier
  ckt_ref = ckt_tgt.copy()
  print("start preprocess")
  syn_preprocess(ckt_tgt)
  syn_preprocess(ckt_goal)
  print("start frontier pushing")
  # if (cg.sat.solve(cg.tx.miter(ckt_ref, ckt_tgt, ckt_ref.inputs(), ckt_ref.outputs()), assumptions={"sat": True}) != False):
  #   cg.to_file(ckt_tgt, "buggy.v")
  #   raise
  # else:
  #   cg.to_file(ckt_tgt, "current.v")

  frontier = list(zip(ckt_tgt.outputs(), ckt_goal.outputs()))
  final_frontier = []
  # for pin_tgt, pin_goal in frontier:
  #   print(pin_tgt, ckt_tgt.fanin_depth(pin_tgt))
  #   print(pin_goal, ckt_goal.fanin_depth(pin_goal))
  print(frontier)
  while (len(frontier) > 0):
    match_nd = frontier[0]
    new_frontier = map_affine(ckt_tgt, ckt_goal, match_nd)

    # print(ckt_tgt.remove_unloaded(inputs = False))
    # ckt_goal.remove_unloaded()
    # print(ckt_tgt.nodes())
    # if (cg.sat.solve(cg.tx.miter(ckt_ref, ckt_tgt, ckt_ref.inputs(), ckt_ref.outputs()), assumptions={"sat": True}) != False):
    #   cg.to_file(ckt_tgt, "buggy.v")
    #   print(truth_to_str(get_truth(ckt_tgt)[0]))
    #   raise
    # else:
    #   cg.to_file(ckt_tgt, "current.v")
    check_ft = frontier.pop(0)
    if (new_frontier == None):
      final_frontier.append(match_nd)
      print("update:", frontier)
      continue
    for pin_tgt, pin_goal in new_frontier:
      if ('dummy' in pin_tgt and 'dummy' in pin_goal): continue
      elif ('dummy' in pin_tgt or 'dummy' in pin_goal): raise
      else: frontier.append((pin_tgt, pin_goal))
    print("update:", frontier)

  # for pin_tgt, pin_goal in final_frontier:
  #   print(pin_tgt, ckt_tgt.fanin_depth(pin_tgt))
  #   print(pin_goal, ckt_goal.fanin_depth(pin_goal))
  print(final_frontier)


In [ ]:
ckt = cg.from_file("adder-mod.v")
ckt_ = cg.from_file("adder-org.v")

ckt_org = ckt.copy()
ckt__org = ckt_.copy()
for o in ckt.outputs():
  ckt.set_output(o, False)
  ckt_org.set_output(o, False)
  ckt_.set_output(o, False)
  ckt__org.set_output(o, False)

ckt.set_output('cOut')
ckt_.set_output('cOut')
ckt_org.set_output('cOut')
ckt__org.set_output('cOut')
ckt.remove_unloaded()
ckt_.remove_unloaded()
print(len(ckt_org.nodes()), len(ckt__org.nodes()))
syn_for_cec(ckt, ckt_)
merge_fanout(ckt)
merge_fanout(ckt_)
ckt.remove_unloaded()
ckt_.remove_unloaded()
remove_buf(ckt)
remove_buf(ckt_)
print(len(ckt.nodes()), len(ckt_.nodes()))
check_ckt = cg.tx.miter(ckt, ckt_org, ckt_org.inputs(), ckt_org.outputs())
check_ckt_ = cg.tx.miter(ckt_, ckt__org, ckt__org.inputs(), ckt__org.outputs())
print(cg.sat.solve(check_ckt, assumptions={"sat": True}))
print(cg.sat.solve(check_ckt_, assumptions={"sat": True}))
cg.to_file(ckt, "circuit_1_cp.v")
cg.to_file(ckt_, "circuit_2_cp.v")


1411 1405
start preprocess
start frontier pushing
[('cOut', 'cOut')]
update: [('_1269_', '_1138_'), ('_1271_', '_1142_'), ('dup285_a_127_', 'dup285_a_127_'), ('dup412_b_127_', 'dup412_b_127_')]
update: [('_1271_', '_1142_'), ('dup285_a_127_', 'dup285_a_127_'), ('dup412_b_127_', 'dup412_b_127_'), ('_1266_', '_0665_'), ('_1268_', '_0793_')]
update: [('dup285_a_127_', 'dup285_a_127_'), ('dup412_b_127_', 'dup412_b_127_'), ('_1266_', '_0665_'), ('_1268_', '_0793_'), ('_0791_', '_1134_'), ('_0919_', '_1141_')]
update: [('dup412_b_127_', 'dup412_b_127_'), ('_1266_', '_0665_'), ('_1268_', '_0793_'), ('_0791_', '_1134_'), ('_0919_', '_1141_')]
update: [('_1266_', '_0665_'), ('_1268_', '_0793_'), ('_0791_', '_1134_'), ('_0919_', '_1141_')]
update: [('_1268_', '_0793_'), ('_0791_', '_1134_'), ('_0919_', '_1141_')]
update: [('_0791_', '_1134_'), ('_0919_', '_1141_')]
update: [('_0919_', '_1141_')]
update: []
[('dup285_a_127_', 'dup285_a_127_'), ('dup412_b_127_', 'dup412_b_127_'), ('_1266_', '_0665

In [ ]:
# gen 4-ary affeq class (it seems to successfully seperate all the functions for 4-ary)
raw_class = dict()
for i in range(2**16):
  truth = dict()
  vec = i
  for j in range(16):
    truth[format(j, f'0{4}b')] = [bool(vec % 2)]
    vec -= vec % 2
    vec /= 2
  # print(truth)
  walsh, auto, onset = check_sig(truth)
  # print(sig_val[0])
  walsh = [abs(val) for val in walsh]
  walsh.sort()
  auto = [abs(val) for val in auto]
  auto.sort()
  sig = str(walsh) + str(auto) + str(onset)
  if (raw_class.get(sig)):
    raw_class[sig].append(format(i, f'0{16}b'))
  else:
    raw_class[sig] = [format(i, f'0{16}b')]
  if (i % 600 == 0): print(".", end = '')
  if (i % 6000 == 0): print("")
# print(raw_class)

with open("raw_class.txt", 'w') as f:
  f.write(str(raw_class))

print(len(raw_class))

.
..........
..........
..........
..........
..........
..........
..........
..........
..........
..........
.........32


In [ ]:
gause = [(x, y) for x in range(4) for y in range(4)]
for x in range(4):
  gause.remove((x,x))

swapping = dict()
for xor in gause:
  swapping[xor] = []
  for i in range(16):
    i_map = i ^ (((i >> (3 - xor[0])) % 2) << (3 - xor[1]))
    swapping[xor].append(i_map)

for selfnot in range(4):
  swapping[tuple([selfnot])] = []
  for i in range(16):
    i_map = i ^ (1 << (3 - selfnot))
    swapping[tuple([selfnot])].append(i_map)
# print(swapping.items())

In [ ]:
affine_lib = dict()
to_syn = []
for key, trs in raw_class.items():
  print("currently:", key)
  trs_cp = set(trs)
  tr_hd = trs[0]
  to_syn.append(tr_hd)
  trs_cp.remove(tr_hd)
  ckt = cg.Circuit()
  for i in range(4):
    ckt.add("PI_{}".format(i), "input")
    ckt.add("PO_{}".format(i), "buf", fanin = ["PI_{}".format(i)], output = True)
  affine_lib[tr_hd] = (tr_hd, ckt)
  trs_tocheck = [tr_hd]
  while (len(trs_tocheck) != 0 and len(trs_cp) != 0):
    tr_checking = trs_tocheck[0]
    for ops, perm in swapping.items():
      # print(perm)
      # print(tr_checking)
      tr_reach = "".join(map(lambda i: tr_checking[i], perm))
      # print(tr_reach)
      if (tr_reach in trs_cp):
        trs_cp.remove(tr_reach)
        trs_tocheck.append(tr_reach)
        pre_ckt = affine_lib[tr_checking][1]
        ckt = pre_ckt.copy()
        if (len(ops) == 1):
          fi = "PI_{}".format(ops[0])
          fo = ckt.fanout(fi)
          # fo = list(ckt.fanout(fi))[0] # should only contain one node
          ckt.disconnect(fi, fo)
          ckt.add("x_{}_{}".format(len(ckt.nodes()) - 8,ops[0]), "not", fanin = [fi], fanout = fo)
        elif (len(ops) == 2):
          # print(ops)
          # fo = "PO_{}".format(ops[1])
          fi_self = "PI_{}".format(ops[1])
          fi_oth = "PI_{}".format(ops[0])
          fo = ckt.fanout(fi_self)
          # print(fi_self, ckt.fanout(fi_self))
          ckt.disconnect(fi_self, fo)
          # print(fo, ckt.fanin(fo))
          ckt.add("x_{}_{}".format(len(ckt.nodes()) - 8,ops[1]), "xor", fanin = [fi_self, fi_oth], fanout = fo)
          # print(fo, ckt.fanin(fo))
        else:
          raise
        affine_lib[tr_reach] = (tr_hd, ckt)
        # if (tr_reach == "0001000000000000"): raise
    # raise
    trs_tocheck.pop(0)
    print("....still {} to work".format(len(trs_cp)))
  if (len(trs_cp) != 0):
    print(trs_cp)
    raise

for trs in to_syn:
  with open("{}.truth".format(trs), "w") as f:
    f.write(to_ABC_truth(trs))


In [ ]:
for f in os.listdir("."):
  if (".v" in f):
    ckt = cg.from_file("{}".format(f))
    affine_lib[f[:-2]] = (f[:-2], ckt)
# print(affine_lib['0000000000010000'])
with open("affine_lib.pkl", "wb") as f:
  pickle.dump(affine_lib, f)

In [ ]:
connect_aff = {'a': 'PO_0', 'b': 'PO_1', 'c': 'PO_2', 'd': 'PO_3'}

for key, val in affine_lib.items():
  tr_top = affine_lib[key][0]
  patch = affine_lib[key][1].copy()

  # tr_top, patch = affine_lib[key]
  # print(key)
  # print(patch)
  # for p in patch: print(p)

  patch_in = sort_pins(patch.inputs())
  # print(patch_in)
  patch_out = sort_pins(patch.outputs())
  # print(patch_out)
  if (tr_top != key):
    # connect = dict(zip(in_ord, affine_lib[tr_top][1]))
    top = affine_lib[tr_top][1].copy()
    # print(cg.io.circuit_to_verilog(top))
    for fo in patch_out:
      patch.set_output(fo, False)
    patch.add_subcircuit(top, 'm', dict(zip(sort_pins(top.inputs()), sort_pins(patch_out))))
    patch.set_output("m_{}".format(list(top.outputs())[0]))
  tr,_ = get_truth(patch)
  # if (key == '0000000000000000'): continue
  if (key != truth_to_str(tr)):
    print(key)
    print(truth_to_str(tr))
    print(cg.io.circuit_to_verilog(affine_lib[key][1]))
    print(cg.io.circuit_to_verilog(top))
    print(cg.io.circuit_to_verilog(patch))
    raise

In [ ]:
# from pyqbf.formula import PCNF
def bmc(v_file_verify, v_file_gold, v_file_trans, log):
  with open(log, 'a') as f:
    i = 0
    t = 0
    while (True):
      obj2id, transPI, id2obj = bmc_step(v_file_verify, v_file_gold, v_file_trans, i)
      # print(id2obj)
      pcnf = PCNF(from_file = "miter.cnf") #, auto_generate_prefix=True)
    # print(pcnf.prefix)
      if (len(obj2id['Ctrl_PI']) == 0):
        pcnf.forall(*(obj2id['PI'])).exists(*(obj2id['Inner']),*(obj2id['miter']))
      else:
        pcnf.exists(*(obj2id['Ctrl_PI'])).forall(*(obj2id['PI'])).exists(*(obj2id['Inner']),*(obj2id['miter'])) # .exists(ctrl).forall(input).exists(inner_sig)
    # print(pcnf.get_block(2))
      # pcnf.to_file("example.qcnf")
      s = Solver(bootstrap_with = pcnf, use_timer = True)
      s.process()
      if (s.solve()):
        t += s.time()
        f.write("solve in {} timeframe with total {} sec\n".format(i,t))
        for var in s.get_model():
          if (var < 0):
            var_name = id2obj[-var]
            var_name = var_name[var_name.find("PI_") + 3:]
            time_frame = var_name.split("_")[0]
            id = var_name.split("_")[1]
            # print(var_name, time_frame, id)
            f.write("!{}({})".format(transPI[int(id)], time_frame))
          else:
            var_name = id2obj[var]
            var_name = var_name[var_name.find("PI_") + 3:]
            time_frame = var_name.split("_")[0]
            id = var_name.split("_")[1]
            # print(var_name, time_frame, id)
            f.write("{}({})".format(transPI[int(id)], time_frame))
        f.write("\n")
        break
      else:
        t += s.time()
        if (s.time() > 60):
          f.write("{} frame run more than 60s\n".format(i))
          break
        elif (i > 12):
          f.write("not equiv with total {} sec\n".format(t))
          break
        else: i+=1

# !touch log.txt
# bmc("cir{}.v".format(1), "cir{}.v".format(129), "ctrlckt_3.v", "log.txt")
# !touch log.txt
# for i in range(1, 255): # (256)
#   for j in range(i, 255): # (i, 256)
#     with open("log.txt", 'a') as f: f.write("{}, {}:\n".format(i,j))
#     bmc("cir{}.v".format(i), "cir{}.v".format(j), "ctrlckt_3.v", "log.txt")

In [ ]:
pcnf = PCNF(from_clauses=[[-1,2],[1,-2]])
pcnf.exists(1).forall(2)
s = Solver(bootstrap_with = pcnf)
s.solve()


False

In [ ]:
# chatgpt -- Runs CAQE solver and extracts Skolem functions from the output

# import subprocess
# import re

# def run_caqe_and_get_skolem(qdimacs_file):
#     """Runs CAQE solver and extracts Skolem functions from the output."""

#     # Run CAQE with Skolem function extraction
#     cmd = ["./caqe", "--qdo", qdimacs_file]
#     result = subprocess.run(cmd, capture_output=True, text=True)

#     # Check for errors
#     if result.returncode != 0:
#         print("Error running CAQE:", result.stderr)
#         return None

#     output = result.stdout
#     skolem_functions = {}

#     # Extract Skolem functions from CAQE output
#     skolem_pattern = re.compile(r"(\d+) = (.+)")

#     for line in output.split("\n"):
#         match = skolem_pattern.match(line.strip())
#         if match:
#             var, expr = match.groups()
#             skolem_functions[int(var)] = expr.strip()

#     return skolem_functions

# # Example usage
# qdimacs_file = "input.qdimacs"  # Replace with your actual QDIMACS file path
# skolem_functions = run_caqe_and_get_skolem(qdimacs_file)

# if skolem_functions:
#     print("Extracted Skolem Functions:")
#     for var, expr in skolem_functions.items():
#         print(f"x{var} = {expr}")


In [ ]:
# https://www.facebook.com/share/r/156wj3JZzK/

In [ ]:
for i in range(64):
  with open("cir{}.truth".format(i), 'w') as f:
    for j in range(8):
      if (i % 2): f.write("1")
      else: f.write("0")
      i /= 2